In [1]:
!pip install -q syncode transformers torch

from syncode import Syncode
import re

MATH_GRAMMAR = """
start: number "," number "," op

number: /-?[0-9]+\\.?[0-9]*/
op: "+" | "-" | "*" | "/"

%import common.WS
%ignore WS
"""

def run_syncode(textual_query: str) -> tuple:
    few_shot_examples = """Extract operands and operator:

Query: What is 327 multiplied by 11.0?
327.0,11.0,*

Query: What is 45.1 plus 23.54?
45.1,23.54,+

Query: What is 120.4 divided by 4.0?
120.4,4.0,/

Query: What is 50 minus 20?
50.0,20.0,-

"""

    syncode = Syncode(
        model="microsoft/phi-2",
        mode="grammar_strict",
        grammar=MATH_GRAMMAR,
        parse_output_only=True
    )

    prompt = few_shot_examples + f"Query: {textual_query}\n"

    print(f"Processing: {textual_query}")

    output = syncode.infer(prompt)[0]

    print(f"Raw output: '{output}'")

    output_clean = output.strip().replace(' ', '')
    parts = output_clean.split(',')

    if len(parts) != 3:
        raise ValueError(f"Unexpected output format: '{output}'. Got {len(parts)} parts instead of 3.")

    operand1 = float(parts[0])
    operand2 = float(parts[1])
    operator = parts[2]

    if operator == '+':
        result = operand1 + operand2
    elif operator == '-':
        result = operand1 - operand2
    elif operator == '*':
        result = operand1 * operand2
    elif operator == '/':
        result = operand1 / operand2
    else:
        raise ValueError(f"Unknown operator: {operator}")

    return ([operand1, operand2], operator, result)


if __name__ == "__main__":
    test_queries = [
        "What is 327 multiplied by 11.0?",
        "What is 45.1 plus 23.54?",
        "What is 120.4 divided by 4.0?"
    ]

    print("Starting tests...\n")

    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}/{len(test_queries)}")
        try:
            operands, operator, answer = run_syncode(query)
            print(f"✓ Operands: {operands}")
            print(f"✓ Operator: {operator}")
            print(f"✓ Final Answer: {answer}")
        except Exception as e:
            print(f"✗ Error: {e}")
        print("-" * 60)


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Starting tests...

Test 1/3
[2025-12-12 20:58:18,214-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

c:\Users\mkaje\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mkaje\OneDrive\Documents\GitHub\CS521-UIUC\HW5\cache\models--microsoft--phi-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✗ Error: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`
------------------------------------------------------------
Test 2/3
[2025-12-12 20:58:18,905-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16
✗ Error: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`
------------------------------------------------------------
Test 3/3
[2025-12-12 20:58:19,000-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16
✗ Error: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`
-------------------------------------